## Long Short-Term Memory Recurrent Neural Network

In [1]:
# Imports
import os, sys
import numpy as np

# Import train_test_split function
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier

# to enable local imports
module_path = os.path.abspath('../code')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

from machine_learning_data_generation import loadOnlineEEGdata

Using TensorFlow backend.


D:\Masterthesis\thesis_eeg\code


## Grid Search LSTM

In [2]:
def create_model():
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# create model
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)



In [ ]:
model

In [2]:
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param)


SyntaxError: unexpected EOF while parsing (<ipython-input-2-cc25b1157c3c>, line 12)

In [3]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 64
    
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
        
    #print("n_timesteps: {} - n_features: {} - n_outputs: {}".format(n_timesteps, n_features, n_outputs))
    model = Sequential()
    
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
    
    
def testModel(trainX, trainy, testX, testy, repeats):
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

def printShapes(trainX, trainy, testX, testy):
    print("Shape train X: {}".format(trainX.shape))
    print("Shape train y: {}".format(trainy.shape))
    print("Shape test X: {}".format(testX.shape))
    print("Shape test y: {}".format(testy.shape))

def runExperiment(data, repeats=5):
    print("Running Experiemtns - Evalute the model {} times".format(repeats))
    trainX, trainy, testX, testy  = data
    printShapes(trainX, trainy, testX, testy)
    testModel(trainX, trainy, testX, testy, repeats)

In [4]:
# load the data
eegData, freqData, entropyData = loadOnlineEEGdata(shuffle=True)

Loading Online EEG Data from D:/Masterthesis/EEG_Data/eeg_data_online ...
EEG Data Shape:
(5024, 512, 40) (5024,) (2154, 512, 40) (2154,)
Freq Data Shape:
(1008, 1, 1200) (1008,) (432, 1, 1200) (432,)
Entropy Data Shape:
(5024, 1, 200) (5024,) (2154, 1, 200) (2154,)


In [6]:
eegX_train, eegy_train, eegX_test, eegy_test = eegData
freqX_train, freqy_train, freqX_test, freqy_test = freqData

In [7]:
freqX_test.shape

(432, 1, 1200)

In [8]:
# Test Frequency Data
runExperiment(freqData, repeats=5)

Running Experiemtns - Evalute the model 5 times
Shape train X: (1008, 1, 1200)
Shape train y: (1008,)
Shape test X: (432, 1, 1200)
Shape test y: (432,)


IndexError: tuple index out of range

In [9]:
# Test EEG Data
runExperiment(eegData, repeats=5)

Running Experiemtns - Evalute the model 5 times
Shape train X: (5024, 512, 40)
Shape train y: (5024,)
Shape test X: (2154, 512, 40)
Shape test y: (2154,)


IndexError: tuple index out of range

#### Slicing up channels

In [ ]:
channel = 6
#trainX = np.dstack(np.dstack(trainX))[channel]
#testX = np.dstack(np.dstack(testX))[channel]
trainX = trainX[:, :, channel:channel+1]
testX = testX[:, :, channel:channel+1]